# MolSysMT Introduction

## Installation

To install MolSysMT, since there is no "stable" version yet, you have to clone the main repository:

```
git clone git@github.com:uibcdf/MolSysMT.git
```

There are some dependencies you have to solve manually (I have to make a file):
numpy, networkx, parmed, mdtraj, nglview, json, os, urllib, pyunitwizard...

Now, with your conda environment activated, install molsysmt as a developing module:

```
python setup.py develop
```

You can now open python, ipython or jupyter to check if molsysmt is imported with out complains:

```
import molsysmt as msm
```

Now, if this happens without errors or warnings, run the NGLView patch to make this library compatible with MolSysMT (This has to be done just the first time):

```
from molsysmt.tools import nglview
nglview.adding_molsysmt()
```

## Tasks to do

In order to solve these tasks, make a copy of this notebook and work over the copy. Name your notebook as you wish, and push it to the main branch of the repository with the cells to do the following: 

- Load the pdb file '1NCR' -from the Protein Data Bank or locally- as a molsysmt.MolSys object.
- Get the form of the new object.
- Print out the information summary of the new object.
- Check if the system has hydrogen atoms. Why there is no hydrogen atoms?
- Remove atoms from water molecules and ions
- Print out the information summary about the molecules of the new object.
- Protonate (add hydrogens) at pH=7.4 to the molecular system.
- Get the number of hydrogen atoms added to the system.
- Get the sequence of aminoacids (3 letters code) of the molecule with index.
- How many atoms with name "CA" has the molecular system?
- Get the list of indices of the groups named "VAL".
- Get the coordinates of the atoms with name "CA" from those groups named "VAL". 
- Show the view of the entity named "W11".
- Show the view of the whole system (water and ions were removed).
- Get the minimum distance between any atom of the small molecule named "W11" and any atom of the molecule with index 0. The same with the molecules indexed 1, and 2.
- Get the distance matrix between the atoms of type "C" from the small molecule named "W11" and any atom named "CA" of molecule with index 0.
- Get all phi and psi dihedral angles of the molecule type peptide.
- Get all covalent chains with the sequence of atoms with name "CB1"-"CB2"-"CB3"-"CB4"-"CB5"-"CB6"-"CB1"
- Get all covalent chains with 7 atoms type "C". How many chains you got? Can this be a bug? If you think so, open and issue reporting this possible bug. Could the method `covalent_chains` be used to detect rings? Should we define a method to get covalent rings?

In [1]:
import molsysmt as msm
import pandas as pd
import nglview as nv

In [2]:
pdb_file = 'pdb:1ncr'
# Fetch file
protein = msm.convert(pdb_file)
print('Form: {}'.format(msm.get_form(protein)))

Form: molsysmt.MolSys


In [3]:
# Print out summary of the system
msm.info(protein)

form,n_atoms,n_groups,n_components,n_chains,n_molecules,n_entities,n_waters,n_ions,n_small_molecules,n_peptides,n_proteins,n_frames
molsysmt.MolSys,6717,1145,344,10,344,7,338,1,2,1,2,1


In [4]:
# Look for hydrogen atoms
n_Hs = msm.get(protein, selection='atom_type=="H"', n_atoms=True)
print('{} hydrogen atoms in the system'.format(n_Hs))

0 hydrogen atoms in the system


In [5]:
# Remove waters
protein = msm.remove_solvent(protein)
msm.info(protein)

form,n_atoms,n_groups,n_components,n_chains,n_molecules,n_entities,n_small_molecules,n_peptides,n_proteins,n_frames
molsysmt.MolSys,6378,806,5,6,5,5,2,1,2,1


In [6]:
# Protonate the molecule
protein = msm.add_missing_hydrogens(protein)
n_Hs = msm.get(protein, selection='atom_type=="H"', n_atoms=True)
print('Number of H atoms after protonation: {}'.format(n_Hs))

Number of H atoms after protonation: 6160


In [7]:
# Sequence of aminoacids with index
aa, ind = msm.get(protein, target="group", group_name=True, group_index=True)
aa_chain = pd.DataFrame(data=aa, index=ind, columns=["Aminoacid"])
print("Protein has {} aminoacids".format(aa_chain.shape[0]))
aa_chain.head(10)

Protein has 806 aminoacids


,Aminoacid
0,ASN
1,PRO
2,VAL
3,GLU
4,ARG
5,TYR
6,VAL
7,ASP
8,GLU
9,VAL


In [10]:
# Atoms with name CA
print("Number of atoms with name CA: {}".format(msm.select(protein, target="atom", selection='atom_name=="CA"').shape[0]))

Number of atoms with name CA: 804


In [11]:
# Indices of VAL
msm.get(protein, target="group", selection="group_name == 'VAL'", index=True)

array([  2,   6,   9,  13,  15,  16,  32,  55,  66,  80,  84, 124, 127,
       138, 144, 167, 208, 209, 221, 229, 231, 232, 253, 269, 273, 284,
       303, 307, 308, 312, 383, 385, 399, 400, 410, 415, 431, 435, 477,
       480, 483, 489, 497, 501, 513, 517, 534, 535, 540, 542, 576, 585,
       601, 606, 623, 665, 689, 690, 693, 699, 702, 703, 734, 735, 749,
       778])

In [12]:
msm.info(protein, target="group", indices=[2,6,9])

index,id,name,type,n atoms,component index,chain index,molecule index,molecule type,entity index,entity name
2,3,VAL,aminoacid,16,0,0,0,protein,0,Protein_0
6,7,VAL,aminoacid,16,0,0,0,protein,0,Protein_0
9,10,VAL,aminoacid,16,0,0,0,protein,0,Protein_0


In [13]:
# Coordinates of the atoms with name "CA" from those groups named "VAL"
msm.get(protein, target="atom", selection='atom_name=="CA" & group_name=="VAL"', coordinates=True)

array([[[ 4.26989985e+00,  1.37030005e+00,  8.99569988e+00],
        [ 4.51170015e+00,  1.84549999e+00,  9.37160015e+00],
        [ 4.76350021e+00,  1.84780002e+00,  9.81900024e+00],
        [ 3.86840010e+00,  2.10890007e+00,  9.94250011e+00],
        [ 3.64770007e+00,  2.69320011e+00,  1.00318003e+01],
        [ 3.47950006e+00,  2.98270011e+00,  1.02094002e+01],
        [ 1.99559999e+00,  4.33879995e+00,  1.11568003e+01],
        [ 3.67479992e+00,  4.02899981e+00,  9.76980019e+00],
        [ 3.65730000e+00,  1.68949997e+00,  1.12403002e+01],
        [ 5.60640001e+00,  2.76600003e-01,  1.34166002e+01],
        [ 6.37979984e+00, -1.19499996e-01,  1.42270002e+01],
        [ 5.73579979e+00,  5.64000010e-01,  1.22678003e+01],
        [ 6.48829985e+00,  2.91999996e-01,  1.28948002e+01],
        [ 6.28520012e+00, -6.34500027e-01,  1.24597998e+01],
        [ 4.37379980e+00, -8.43900025e-01,  1.26983004e+01],
        [ 5.34969997e+00, -6.38199985e-01,  1.19961996e+01],
        [ 3.23779988e+00,  3.80400002e-01,  1.32944002e+01],
        [ 2.92980003e+00,  1.69200003e-01,  1.32233000e+01],
        [ 6.06339979e+00, -9.82100010e-01,  1.28015003e+01],
        [ 6.64080000e+00,  6.09999988e-03,  1.34533997e+01],
        [ 6.12449980e+00,  1.72099993e-01,  1.31307001e+01],
        [ 5.85500002e+00,  4.29600000e-01,  1.30669003e+01],
        [ 3.72210002e+00,  2.34540009e+00,  1.34749002e+01],
        [ 3.56139994e+00,  3.03160000e+00,  1.41801996e+01],
        [ 3.78169990e+00,  2.89010000e+00,  1.35289001e+01],
        [ 3.89720011e+00,  4.66499996e+00,  1.27639999e+01],
        [ 5.58300018e-01,  3.20519996e+00,  1.20277996e+01],
        [ 1.28040004e+00,  2.43840003e+00,  1.17152996e+01],
        [ 1.34249997e+00,  2.09540009e+00,  1.15654001e+01],
        [ 1.55980003e+00,  1.59200001e+00,  1.12356997e+01],
        [ 8.44799995e-01,  1.98880005e+00,  1.24633999e+01],
        [ 7.22400010e-01,  2.52189994e+00,  1.28052998e+01],
        [ 1.51160002e+00,  2.66179991e+00,  1.27712002e+01],
        [ 1.74600005e+00,  2.38190007e+00,  1.26660995e+01],
        [ 3.13290000e+00,  1.87170005e+00,  1.43400002e+01],
        [ 3.15630007e+00,  9.59100008e-01,  1.43765001e+01],
        [ 1.81659997e+00,  2.42899990e+00,  1.41752005e+01],
        [ 1.97570002e+00,  1.97290003e+00,  1.51421003e+01],
        [ 1.35090005e+00,  1.65349996e+00,  1.19382000e+01],
        [ 1.92509997e+00,  1.05369997e+00,  1.19792995e+01],
        [ 2.20000005e+00,  2.49300003e-01,  1.19844999e+01],
        [ 1.46249998e+00,  2.88899988e-01,  1.33149996e+01],
        [ 1.71190000e+00,  2.25169992e+00,  1.31574001e+01],
        [ 1.92750001e+00,  3.42020011e+00,  1.32842999e+01],
        [ 4.19499993e-01,  3.35999990e+00,  1.37816000e+01],
        [ 5.86000025e-01,  2.19930005e+00,  1.31173000e+01],
        [ 9.60300028e-01, -1.90599993e-01,  1.39899998e+01],
        [ 1.22669995e+00,  6.32999986e-02,  1.40866003e+01],
        [ 6.69029999e+00,  1.84599996e-01,  9.84860039e+00],
        [ 6.68690014e+00,  5.59499979e-01,  1.03619003e+01],
        [ 2.91120005e+00,  1.76250005e+00,  1.12718000e+01],
        [ 2.61199999e+00,  2.92309999e+00,  1.16320000e+01],
        [ 2.44770002e+00,  3.41129994e+00,  1.35750999e+01],
        [ 2.44400001e+00,  4.40619993e+00,  1.31099997e+01],
        [ 3.70029998e+00,  4.24189997e+00,  1.19242001e+01],
        [ 2.14510012e+00,  5.24340010e+00,  1.21695004e+01],
        [ 2.45269990e+00,  5.02220011e+00,  1.14151001e+01],
        [ 2.18460011e+00,  5.22949982e+00,  1.15893002e+01],
        [ 1.42379999e+00,  4.84600019e+00,  1.20834999e+01],
        [ 1.98619998e+00,  4.33199978e+00,  1.16980000e+01],
        [ 2.87700009e+00,  4.28079987e+00,  1.12203999e+01],
        [ 3.24609995e+00,  4.34770012e+00,  1.12803001e+01],
        [ 1.23740005e+00,  5.51380014e+00,  1.28993998e+01],
        [ 9.50500011e-01,  5.34950018e+00,  1.30847998e+01],
        [ 2.86789989e+00,  3.60990000e+00,  1.19271002e+01],
        [ 4.47480011e+00,  4.429000

In [14]:
msm.info(protein, target='component')

index,n atoms,n groups,chain index,molecule index,molecule type,entity index,entity name
0,4489,285,0,0,protein,0,Protein_0
1,7571,490,1,1,protein,1,Protein_1
2,436,29,3,2,peptide,2,Peptide_0
3,27,1,4,3,small molecule,3,W11
4,15,1,5,4,small molecule,4,MYR


In [62]:
# Taking a look at the system

view = msm.view(protein, viewer="NGLView")
view.add_ball_and_stick(w11_molecule, color="orange")
view

NGLWidget()

In [ ]:
# View of the entity named "W11"
w11_molecule = msm.select(protein, selection='component_index==3', to_syntaxis="NGLView")
w11_view = msm.view(w11_molecule, viewer="NGLView")

In [44]:
# Get the minimum distance between any atom of the small molecule named "W11" and any atom of the molecule with index 0. The same with the molecules indexed 1, and 2

w11_atom_index = msm.get(protein, target='group', selection='component_index==3', atom_index=True)

print(f"Small molecule shape: {w11_atom_index.shape}\n")

for i in range(3):
    protein_atom_index = msm.get(protein, target='group', selection='component_index=={}'.format(i), atom_index=True)
    min_pairs, min_distances = msm.minimum_distance(protein,
                                                    groups_of_atoms=protein_atom_index,
                                                    group_behavior='geometric_center',
                                                    groups_of_atoms_2=w11_atom_index,
                                                    group_behavior_2='geometric_center')
    
    print(f"Molecule number {i}\n\nShape: {protein_atom_index.shape}")
    print(f"Min pairs: {min_pairs[0]}")
    print(f"Min distance: {min_distances[0]}\n\n")

Small molecule shape: (1, 27)

Molecule number 0

Shape: (285,)
Min pairs: [97  0]
Min distance: 0.5041120436060018 nanometer


Molecule number 1

Shape: (490,)
Min pairs: [275   0]
Min distance: 0.882876145000038 nanometer


Molecule number 2

Shape: (29,)
Min pairs: [20  0]
Min distance: 1.2653449156135292 nanometer




In [52]:
# Get the distance matrix between the atoms of type "C" from the small molecule named "W11" and any atom named "CA" of molecule with index 0
protein_atom_index = msm.get(protein, target='group', selection='component_index==0 & atom_name=="CA"', atom_index=True)
w11_atom_index = msm.get(protein, target='group', selection='component_index==3 & atom_type=="C"', atom_index=True)

neighbors, distances = msm.neighbors(protein,
                                    groups_of_atoms=protein_atom_index,
                                    group_behavior='geometric_center',
                                    groups_of_atoms_2=w11_atom_index,
                                    group_behavior_2='geometric_center',
                                    num_neighbors=1)

distances.shape

(1, 285, 1)

In [67]:
# Get all phi and psi dihedral angles of the molecule type peptide
covalent_chains = msm.covalent_dihedral_quartets(protein, dihedral_angle="phi-psi", selection='molecule_type=="peptide"')
dihedral_angles = msm.get_dihedral_angles(protein, quartets=covalent_chains)
dihedral_angles.shape

(1, 56)

In [8]:
# Get all covalent chains with the sequence of atoms with name "CB1"-"CB2"-"CB3"-"CB4"-"CB5"-"CB6"-"CB1"
chain = ["C1B", "C2B", "C3B", "C4B", "C5B", "C6B", "C1B"]

for j in range(len(chain)):
    chain[j] = 'atom_name=="{}"'.format(chain[j])
    
msm.covalent_chains(protein, chain=chain)

array([[12506, 12507, 12509, 12510, 12511, 12512, 12506]])

In [9]:
msm.select(protein, target="atom", selection='atom_name=="C1B"')

array([12506])

In [10]:
# Get all covalent chains with 7 atoms type "C"
chain = ['atom_type=="C"'] * 7
msm.covalent_chains(protein, chain=chain)

array([[    4,     6,     4, ...,     4,     6,     4],
       [    4,     6,     4, ...,     4,     8,     4],
       [    4,     6,     4, ...,     4,     8,    11],
       ...,
       [12537, 12536, 12537, ..., 12535, 12536, 12537],
       [12537, 12536, 12537, ..., 12537, 12536, 12535],
       [12537, 12536, 12537, ..., 12537, 12536, 12537]])